<a href="https://colab.research.google.com/github/LukeTheGeneWriter/gene_system_pHMMer/blob/main/gene_system_pHMMer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Luke Williams -- 2/8/2025

Welcome to the gene system pHMMer. This script works by walking through a gene file system and creating a profile hidden markov model of the sequences found in each gene folder.

This allows follow-on analysis from the file system generated by AutoClipper, found at https://github.com/LukeTheGeneWriter/AutoClipper

This script is pretty plug-and-play, but you do need to edit some path variables so that the program knows where the genetic file system is located, where it can write temporary files, and where it should output the final pHMM.

Setup environment

In [ ]:
!pip install -q condacolab
!pip install biopython
import condacolab
condacolab.install()
!conda install -c bioconda pyhmmer
!conda install bioconda::clustalw

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
%ls
%cd drive
%cd MyDrive
%ls

import os
import pyhmmer
import numpy as np
from Bio import SeqIO
from Bio import Seq
from Bio import AlignIO
from Bio.SeqRecord import SeqRecord

!!! You'll need to set these paths to connect this program to the data you want to process. Set the dest, interdest, and genesysdest variables.

In [ ]:
#Set the destination folder for the profile hidden markov models
dest = 'Gene_Sysyem_HMMs'
if not os.path.exists(dest):
  os.makedirs(dest)

#Set the intermediate folder for gene multi-seuence alignments
interdest = 'Genes_Aligned'
if not os.path.exists(interdest):
  os.makedirs(interdest)

#Set the gene system folder
genesysdest = 'Genomes_By_Gene'

genes = os.listdir(genesysdest)
print(genes)

Below is a function to format the sequence records since Clustal Omega does not tolerate duplicate names

In [ ]:
def rename_dups(g):
  finrecs = []
  seen_names = []
  seen_ids = []
  for record in SeqIO.parse(os.path.join(interdest, g), 'fasta'):
    #check if we'e seen it before
    if record.name in seen_names or record.id in seen_ids:
      #Check how many times it appears in the list
      n = 0
      for i in range(0, len(seen_names)):
        if record.name == seen_names[i] or record.id == seen_ids[i]:
          n += 1
      #create a new record with "__" + str(n) appended to the rec id and name
      newrec = SeqRecord(record.seq, id = record.id + "__" + str(n), name = record.name + "__" + str(n), description = record.description)
      seen_names.append(record.name)
      seen_ids.append(record.id)
      finrecs.append(newrec)
    else:
      newrec = SeqRecord(record.seq, id = record.id + "__0", name = record.name + "__0", description = record.description)
      finrecs.append(newrec)
      seen_names.append(record.name)
      seen_ids.append(record.id)
  #print(seen_names)
  #print(seen_ids)
  return finrecs


This is the main

In [ ]:
for g in genes:
  #Skip genes which are already computed
  if os.path.exists(os.path.join(dest, g + "_hmm")):
    print("Skipping " + g)
    continue

  #Aggregate the fastas into one fasta
  with open(os.path.join(interdest, g), 'w') as fyle:
    for strain_file in os.listdir(os.path.join(genesysdest, g)):
      strain_file_path = os.path.join(genesysest, g, strain_file)
      for record in SeqIO.parse(strain_file_path, 'fasta'):
        SeqIO.write(record, fyle, 'fasta')

  #Set clustalw paths
  clustal_in = os.path.join(interdest, g + "_prepped")
  clustal_out = os.path.join(interdest, g + "_alig")

  #Rename duplicates -- clustalw does not tolerate multiple seqs with the same name
  non_dup_recs = rename_dups(g)

  #Write one big fasta to put into clustal
  with open(clustal_in, 'w') as c_in:
    for r in non_dup_recs:
      SeqIO.write(r, c_in, 'fasta')

  #Run Clustal Omega to align the sequences
  !clustalw -INFILE={clustal_in} -OUTFILE={clustal_out}

  #Create pHMM from MSA
  hmm_in = clustal_out
  hmm_out = os.path.join(dest, g + "_hmm")
  alphabet = pyhmmer.easel.Alphabet.dna()

  with pyhmmer.easel.MSAFile(hmm_in, digital=True, alphabet=alphabet) as msa_file:
    msa = msa_file.read()
    interstring = g + "_msa"
    msa.name = interstring.encode()
    builder = pyhmmer.plan7.Builder(alphabet)
    background = pyhmmer.plan7.Background(alphabet)
    hmm, _, _ = builder.build_msa(msa, background)
    hmm.consensus
    #save
    with open(hmm_out, 'wb') as hmmout_file:
      hmm.write(hmmout_file)